In [ ]:
import pandas as pd
import warnings
import os

warnings.filterwarnings("ignore")   

base_pagos_efect_path = 'bases/efectividades/BasePagos_Efectividades_20240919_Contrato.xlsx'
df_pagos = pd.read_excel(base_pagos_efect_path, sheet_name="BD")

dashboard_efect_path = 'dashboard/DASHBOARD_EFECTIVIDADES_SEP19.xlsx'
dashboard_saldos_path = 'dashboard/DASHBOARD_SALDOS_SEP19.xlsx'
dashboard_indicadores_path = 'dashboard/DASHBOARD_INDICADORES_SEP19.xlsx'

dashboard_efect_path = os.path.abspath(dashboard_efect_path)
dashboard_saldos_path = os.path.abspath(dashboard_saldos_path)
dashboard_indicadores_path = os.path.abspath(dashboard_indicadores_path)

print(dashboard_efect_path)
print(dashboard_saldos_path)
print(dashboard_indicadores_path)

In [ ]:
df_pagos_efect = df_pagos.copy()
df_pagos_saldos = df_pagos.copy()
df_pagos_indicadores = df_pagos.copy()

print(df_pagos.shape)
df_pagos.head(3)

In [ ]:
cols =  ["fecha", "periodo", "dia", "Dia_Util", "codcent", "contrato", "clave", "CapitalSoles", "PagoTotalSoles", "PagoEfectTotalSoles", "PagoEfectTotalSolesAgencia", "PagoEfectTotalSolesAgenciaCont", "Agencia", "desc_tramo", "foco"]
df_pagos_efect = df_pagos_efect[cols]
print(df_pagos_efect.shape)

df_pagos_efect = df_pagos_efect[df_pagos_efect["PagoEfectTotalSolesAgenciaCont"]!=0]
print(df_pagos_efect.shape)

def marca_kst(row):
    if row["clave"] == "KSTBC":
        if row["desc_tramo"] in ["< 01 año", "De 90 a 180 dias", "< 02 años"]:
            return "KSTBC"
        else:
            return "KSTBC_2"
    return ""

df_pagos_efect["MARCA KST"] = df_pagos_efect.apply(marca_kst, axis=1)
df_pagos_efect["codcent"] = df_pagos_efect["codcent"].astype(str)
df_pagos_efect["contrato"] = df_pagos_efect["contrato"].astype(str).str.zfill(20)
df_pagos_efect["cont_18"] = df_pagos_efect["contrato"].apply(lambda x: x[:8] + x[-10:])

cols =  ["fecha", "periodo", "dia", "Dia_Util", "codcent", "contrato", "cont_18", "clave", "CapitalSoles", "PagoTotalSoles", "PagoEfectTotalSoles", "PagoEfectTotalSolesAgencia", "PagoEfectTotalSolesAgenciaCont", "Agencia", "foco", "MARCA KST"]
df_pagos_efect = df_pagos_efect[cols]
df_pagos_efect.to_excel(dashboard_efect_path, index=False)

df_pagos_efect.head(3)

In [4]:
###################################################################################################################

In [ ]:
cols =  ['Agencia', 'clave', 'SEGMENTO_RIESGO', 'AMBITO_RCD_FINAL', 'CapitalSoles']
df_pagos_saldos = df_pagos_saldos[cols]
print(df_pagos_saldos.shape)

df_pagos_saldos.rename(columns={'clave': 'CLAVE', 'Agencia': 'AGENCIA', 'CapitalSoles': 'CAPITAL', 'AMBITO_RCD_FINAL': 'PRODUCTO', 'SEGMENTO_RIESGO': 'SEGMENTO'}, inplace=True)
print(df_pagos_saldos.shape)

df_pagos_saldos = df_pagos_saldos[df_pagos_saldos['AGENCIA'].isin(["ASESCOM RJ", "CLASA MORA", "MORNESE MORA", "SIN AGENCIA"])]

df_pagos_saldos['SEGMENTO'] = df_pagos_saldos['SEGMENTO'].apply(lambda x: 'PYME / EMP MIN' if x != 'PARTICULARES' else x)
df_pagos_saldos['PRODUCTO'] = df_pagos_saldos['PRODUCTO'].str.replace(r'\d+', '', regex=True).str.replace('.', '').str.upper()
df_pagos_saldos['PRODUCTO'] = df_pagos_saldos['PRODUCTO'].str.strip().replace(' ', '')
df_pagos_saldos['CAPITAL'] = df_pagos_saldos['CAPITAL'].round(2)

df_pagos_saldos = df_pagos_saldos.groupby(['AGENCIA', 'CLAVE', 'SEGMENTO', 'PRODUCTO']).agg({'CAPITAL': 'sum'}).reset_index()
df_pagos_saldos.to_excel(dashboard_saldos_path, index=False)

df_pagos_saldos.head(3)

In [6]:
###################################################################################################################

In [ ]:
cols =  ['Agencia', 'clave', 'Intensidad', 'Contacto_Efectivo', 'Tasa_Cierre', 'PDP_Cumplida']
df_pagos_indicadores = df_pagos_indicadores[cols]
print(df_pagos_indicadores.shape)

df_pagos_indicadores.rename(columns={
    'Agencia': 'AGENCIA', 
    'clave': 'CLAVE', 
    'Intensidad': 'INTENSIDAD TOTAL', 
    'Contacto_Efectivo': 'CONTACTABILIDAD DIRECTA', 
    'Tasa_Cierre': 'TASA CIERRE', 
    'PDP_Cumplida': 'CALIDAD PROMESAS'
    }, inplace=True)

df_pagos_indicadores = df_pagos_indicadores[df_pagos_indicadores['AGENCIA'].isin(["ASESCOM RJ", "CLASA MORA", "MORNESE MORA"])]

df_pagos_indicadores['INTENSIDAD TOTAL'] = df_pagos_indicadores['INTENSIDAD TOTAL'].fillna(0)
df_pagos_indicadores['CONTACTABILIDAD DIRECTA'] = df_pagos_indicadores['CONTACTABILIDAD DIRECTA'].fillna('NULL')
df_pagos_indicadores['TASA CIERRE'] = df_pagos_indicadores['TASA CIERRE'].fillna('NULL')
df_pagos_indicadores['CALIDAD PROMESAS'] = df_pagos_indicadores['CALIDAD PROMESAS'].fillna('NULL')
# convierte 'NULL' en NaN
df_pagos_indicadores['CONTACTABILIDAD DIRECTA'] = pd.to_numeric(df_pagos_indicadores['CONTACTABILIDAD DIRECTA'], errors='coerce')
df_pagos_indicadores['TASA CIERRE'] = pd.to_numeric(df_pagos_indicadores['TASA CIERRE'], errors='coerce')
df_pagos_indicadores['CALIDAD PROMESAS'] = pd.to_numeric(df_pagos_indicadores['CALIDAD PROMESAS'], errors='coerce')

df_pagos_indicadores = df_pagos_indicadores.groupby(['AGENCIA', 'CLAVE']).agg({
    'INTENSIDAD TOTAL': 'mean', 
    'CONTACTABILIDAD DIRECTA': lambda x: x.mean(skipna=True), 
    'TASA CIERRE': lambda x: x.mean(skipna=True), 
    'CALIDAD PROMESAS': lambda x: x.mean(skipna=True)
    }).reset_index()

df_pagos_indicadores['INTENSIDAD TOTAL'] = df_pagos_indicadores['INTENSIDAD TOTAL'].round(2)
df_pagos_indicadores['CONTACTABILIDAD DIRECTA'] = df_pagos_indicadores['CONTACTABILIDAD DIRECTA'].round(2)
df_pagos_indicadores['TASA CIERRE'] = df_pagos_indicadores['TASA CIERRE'].round(2)
df_pagos_indicadores['CALIDAD PROMESAS'] = df_pagos_indicadores['CALIDAD PROMESAS'].round(2)

df_pagos_indicadores.to_excel(dashboard_saldos_path, index=False)

df_pagos_indicadores.head(3)

In [7]:
os.startfile(dashboard_efect_path)
os.startfile(dashboard_saldos_path)
os.startfile(dashboard_indicadores_path)